##### 보스톤 주택 가격 데이터셋으로 네트워크를 이용한 회귀 분석

In [4]:
# Boston Housing Dataset는 1978년 미국 보스톤 지역의 주택 가격으로 506개 타운의 주택 가격 중앙값을 
# 1000달라 단위로 나타내고 있다. 범죄율, 주택당 방개수, 고속도로까지의 거리 등 13가지 데이터를 이용해서 
# 주택가격을 예측해야 한다.
#
# 훈련데이터로 학습할 때 일부 데이터를 떼어내서 검증(validation)데이터로 만들 수 있다. 학습에 쓰이는 데이터는 
# 훈련데이터, 검증에 쓰이는 데이터는 검증데이터, 최종 평가에 쓰이는 데이터는 테스트데이터이다.
# 이 가운데 딥러닝 네트워크의 가중치에 영향을 주는 데이터는 훈련데이터뿐임을 기억하자!!!
# 훈련데이터의 일부를 떼어서 검증데이터로 만들어야 하는데 보통 전체 데이터의 20% 정도를 사용한다.
# 즉 훈련, 검증, 테스트 데이터의 비율은 약 6:2:2 정도가 된다.

from tensorflow.keras.datasets import boston_housing

(train_X, train_Y), (test_X, test_Y) = boston_housing.load_data()
print(len(train_X), len(test_X))
print(train_X[0])
print()
print(train_Y[0])

404 102
[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]

15.2


In [6]:
# 보스톤 주택 가격 데이터세트의 각 데이터 속성은 다음과 같다.
# CRIM(범죄율), ZN(25,000평방피트당 주거지역의 비율), INDUS(비소매 상업지구 비율(단위:에이커)), 
# CHAS(찰스강에 인접해 있으면 1, 그렇지 않으면 0), NOX(일산화질소 농도(단위:0.1ppm)), RM(주택당 방의 수), 
# AGE(1940년 이전에 건설된 주택의 비율), DIS(5개의 보스톤 직업고용센터와의 거리(가중 평균)), RAD(고속도로 접근성), 
# TAX(재산세율), PTRATIO(학생/교사 비율), B(흑인 비율), LSTAT(하위계층비율), MEDV(타운의 주택가격 중앙값(1000달라))

# 위 설명에서 주목해야 할 점은 각 데이터의 단위가 다르다는 것이다. 비율(%)인 것도 있고, 0/1로 나타내는 데이터,
# 양의 정수등 다양하다. 실생활에서 얻는 데이터는 이처럼 다양한 단위를 가지고 있는 경우가 많다. 
# 딥러닝에서는 이러한 데이터를 전처리해서 졍규화해야 학습 효율이 좋다.(표준정규분포로 옮긴다)

x_mean = train_X.mean()
x_std = train_X.std()
y_mean = train_Y.mean()
y_std = train_Y.std()

train_X = (train_X - x_mean)/x_std
test_X = (test_X - x_mean)/x_std
    
train_Y = (train_Y - y_mean)/y_std
test_Y = (test_Y - y_mean)/y_std

print(train_X[0])
print()
print(train_Y[0])

[-0.47482083 -0.48335641 -0.42698208 -0.48335641 -0.47963044 -0.44081941
  0.15172056 -0.45581402 -0.45565404  1.64280094 -0.33791894  2.26541184
 -0.35370929]

-0.7821526033779157


In [7]:
# 계속해서 Sequential Model을 사용해서 네트워크를 구성하겠다.
import tensorflow as tf

In [9]:
# 속성의 수도 13개로 많고 데이터 양도 많기 때문에 4개의 레이어를 사용했고 각 레이어에 들어가는 뉴런의 수도 늘렸다.
# 레이어수와 은닉층의 뉴런수를 늘리면 모델의 표현력이 좋아지기 때문에 더 복잡한 내용을 학습시킬 수 있다.
# 지금처럼 여러개의 레이어를 겹쳐서 사용할 때는 시그모이드, tanh보다 relu가 더 좋은 결과를 얻을 수 있다.

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=52, activation='relu', input_shape=(13, )),
    tf.keras.layers.Dense(units=39, activation='relu'),
    tf.keras.layers.Dense(units=26, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.07), loss='mse')

model.summary()   # 14*52=728, 53*39=2067, 40*26=1040, 27*1=27

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                728       
                                                                 
 dense_1 (Dense)             (None, 39)                2067      
                                                                 
 dense_2 (Dense)             (None, 26)                1040      
                                                                 
 dense_3 (Dense)             (None, 1)                 27        
                                                                 
Total params: 3,862
Trainable params: 3,862
Non-trainable params: 0
_________________________________________________________________
